In [1]:
param_make = 'ACURA'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./' + param_make + '/5.category.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_make = df['Make'].tolist()
list_year = df['Year'].tolist()
list_model = df['Model'].tolist()
list_engine = df['Engine'].tolist()
list_car_code = df['Car_Code'].tolist()
list_category = df['Category'].tolist()
list_category_code = df['Category_Code'].tolist()

print('总数量：' + str(len(list_category)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for make, year, model, engine, car_code, category, category_code in zip(list_make, list_year, list_model, list_engine, list_car_code, list_category, list_category_code):
    work.put_nowait((make, year, model, engine, car_code, category, category_code))

def crawler():
    global df
    
    while not work.empty():
        make, year, model, engine, car_code, category, category_code = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                data = 'func=navnode_fetch&payload=' + quote('{"jsn":{"tab":"catalog","make":"') + make.replace(' ', '+') + quote('","year":"') + year + quote('","model":"') + model.replace('+', '%2B').replace(' ', '+').replace('&', '%26').replace('/', '%2F') + quote('","engine":"') + engine.replace(' ', '+').replace('/', '%2F') + quote('","carcode":"') + car_code + quote('","groupname":"') + category.replace(' ', '+').replace('&', '%26').replace('/', '%2F') + quote('","nodetype":"groupname"}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'},
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                if html == '':
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Make': make,
                                             'Year': year,
                                             'Model': model,
                                             'Engine': engine,
                                             'Car_Code': car_code,
                                             'Category': category,
                                             'Category_Code': category_code,
                                             'Type': 'Nan',
                                             'Type_Code': 'Nan'}])
                    status = 'ok'
                    
                    break
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_type = html.xpath('//td[@class="nlabel"]/a/text()')

                # = = = = = = = = = = = = = = =
                
                list_url = html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')

                list_type_code = [_.split(',')[-1] for _ in list_url]

                # = = = = = = = = = = = = = = =

                list_url = ['https://www.rockauto.com'+_ for _ in list_url]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Make': make,
                                        'Year': year,
                                        'Model': model,
                                        'Engine': engine,
                                        'Car_Code': car_code,
                                        'Category': category,
                                        'Category_Code': category_code,
                                        'Type': list_type,
                                        'Type_Code': list_type_code,
                                        'Url': list_url})

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Make': make,
                                     'Year': year,
                                     'Model': model,
                                     'Engine': engine,
                                     'Car_Code': car_code,
                                     'Category': category,
                                     'Category_Code': category_code}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(make + ', ' + year + ', ' + model + ', ' + engine + ', ' + category +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./' + param_make + '/type_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：1

ACURA, 2020, TLX, 3.5L V6 DOHC, Wiper & Washer  <->  [ok] - 剩余数量：0

爬虫输出中

爬虫结束
